In [82]:
import pandas as pd
import glob

# Data import

In [83]:
df = pd.read_csv('data/NBA_Player_Dataset-2013-2021.csv')

box_cox_transformer = 0.16106145323461019 #need to pull from data_prep.ipynb

# Scale Data

In [156]:
#First scaling 2021 season to a full season
#roughly 10 games left for all teams out of 82

scale_factor = 1 + 10/82

#columns to scale

cols_to_scale = ['GP', 'FTA', '2PA', '3PA']

df[cols_to_scale] = df[cols_to_scale] * scale_factor

In [157]:
#variables we need to scale

x_to_scale = df[['AGE', 'GP', 'MPG', 'MIN%', 'USG%',
       'FTA', 'FT%', '2PA', '2P%', '3PA', '3P%', 'eFG%', 'TS%', 'PPG', 'RPG',
       'TRB%', 'APG', 'AST%', 'SPG', 'BPG', 'VI', 'TO_100_Games']]

#scaler function
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(x_to_scale)

x_scaled = scaler.transform(x_to_scale)

x_scaled_df = pd.DataFrame(x_scaled, columns = x_to_scale.columns)

In [158]:
#add the categorical features to scaled features for input
X_with_year = pd.concat([x_scaled_df, df[['Year', 'Center', 'Forward', 'Guard']]], axis = 1)

#choose our target variable
#y = df['Salary']
y = df['Salary_BoxCox']

# Read in results from models_final.ipynb

In [159]:
df_list = []

for file in glob.glob('results/*.csv'):
    temp_df = pd.read_csv(file)
    df_list.append(temp_df)

results_df = pd.concat(df_list, axis = 0)

In [160]:
#Remove the columns specific to certain models, all parameters are stored in 'params' column

results_df = results_df[['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'params', 
                        'mean_test_score', 'std_test_score', 'rank_test_score', 'Model Type', 'Training Data']]

In [161]:
#so far best model is RF with full dataset.  
results_df.sort_values('mean_test_score', ascending = True).to_csv('model_results.csv', index = False)
results_df.sort_values('mean_test_score', ascending = True).head(15)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,params,mean_test_score,std_test_score,rank_test_score,Model Type,Training Data
9,6.598142,0.402557,0.102035,0.010738,"{'max_depth': 25, 'max_features': 10, 'n_estim...",9.155130,0.215472,1,Random Forest,Full Dataset
11,9.665229,0.637191,0.075954,0.006360,"{'max_depth': 25, 'max_features': 20, 'n_estim...",9.174885,0.193764,2,Random Forest,Full Dataset
3,4.772411,0.690266,0.084805,0.012553,"{'max_depth': 10, 'max_features': 10, 'n_estim...",9.193707,0.231902,3,Random Forest,Full Dataset
8,2.738060,0.348547,0.049800,0.013287,"{'max_depth': 25, 'max_features': 10, 'n_estim...",9.195151,0.178750,4,Random Forest,Full Dataset
4,3.501711,0.512405,0.034989,0.001272,"{'max_depth': 10, 'max_features': 20, 'n_estim...",9.204922,0.226176,5,Random Forest,Full Dataset
5,9.083368,1.173974,0.094001,0.035769,"{'max_depth': 10, 'max_features': 20, 'n_estim...",9.207776,0.194854,6,Random Forest,Full Dataset
2,2.037275,0.327192,0.038315,0.008223,"{'max_depth': 10, 'max_features': 10, 'n_estim...",9.211908,0.247775,7,Random Forest,Full Dataset
7,4.118319,0.531390,0.118671,0.022865,"{'max_depth': 25, 'max_features': 5, 'n_estima...",9.227281,0.207246,8,Random Forest,Full Dataset
10,4.781488,0.240398,0.044402,0.004756,"{'max_depth': 25, 'max_features': 20, 'n_estim...",9.252220,0.204945,9,Random Forest,Full Dataset
6,1.541104,0.043050,0.044734,0.004706,"{'max_depth': 25, 'max_features': 5, 'n_estima...",9.275502,0.217739,10,Random Forest,Full Dataset


In [162]:
#worst models
results_df.sort_values('mean_test_score', ascending = True).tail(15)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,params,mean_test_score,std_test_score,rank_test_score,Model Type,Training Data
41,1.368311,0.016403,0.278649,0.012914,"{'C': 1, 'gamma': 'scale', 'kernel': 'sigmoid'}",50.436944,4.067735,57,SVM,Full Dataset
44,1.268169,0.042970,0.252600,0.022811,"{'C': 1, 'gamma': 'auto', 'kernel': 'sigmoid'}",100.884112,5.798606,56,SVM,Subset Dataset
41,1.272010,0.061068,0.250168,0.016202,"{'C': 1, 'gamma': 'scale', 'kernel': 'sigmoid'}",100.918061,6.711417,57,SVM,Subset Dataset
50,1.371754,0.026226,0.280721,0.013525,"{'C': 10, 'gamma': 'auto', 'kernel': 'sigmoid'}",463.562443,36.662448,58,SVM,Full Dataset
47,1.368837,0.053628,0.267229,0.007819,"{'C': 10, 'gamma': 'scale', 'kernel': 'sigmoid'}",520.176297,46.029433,59,SVM,Full Dataset
47,1.372343,0.088790,0.265795,0.008835,"{'C': 10, 'gamma': 'scale', 'kernel': 'sigmoid'}",1002.214085,67.673235,58,SVM,Subset Dataset
50,1.411286,0.124818,0.263104,0.005914,"{'C': 10, 'gamma': 'auto', 'kernel': 'sigmoid'}",1002.429532,59.529284,59,SVM,Subset Dataset
56,1.385706,0.047284,0.277602,0.005083,"{'C': 100, 'gamma': 'auto', 'kernel': 'sigmoid'}",3913.287950,685.142338,60,SVM,Full Dataset
53,1.406578,0.056377,0.272704,0.009103,"{'C': 100, 'gamma': 'scale', 'kernel': 'sigmoid'}",4729.386173,510.797848,61,SVM,Full Dataset
56,1.341692,0.035011,0.257934,0.007764,"{'C': 100, 'gamma': 'auto', 'kernel': 'sigmoid'}",10009.943693,588.147454,60,SVM,Subset Dataset


In [163]:
#parameters for best model
best_params = results_df.sort_values('mean_test_score', ascending = True).iloc[0]['params']
best_params

"{'max_depth': 25, 'max_features': 10, 'n_estimators': 250}"

# Train model on previous years, predict on 2021 season

In [164]:
X_train = X_with_year[X_with_year['Year'] != 2021].drop('Year', axis = 1)
y_train = df['Salary_BoxCox'][df['Year'] != 2021]

X_test = X_with_year[X_with_year['Year'] == 2021].drop('Year', axis = 1)
y_test = df['Salary_BoxCox'][df['Year'] == 2021]

In [165]:
X_train[cols_to_scale]

,GP,FTA,2PA,3PA
537,0.485019,0.023672,-0.168080,-0.912200
538,-1.714062,-0.894620,-1.012270,-0.905284
539,0.362847,0.232375,0.129870,-0.898367
540,0.607190,2.060612,1.990144,-0.863785
541,-0.940312,-0.502259,-0.210098,-0.365804
...,...,...,...,...
4205,0.932979,1.826865,1.333128,0.381168
4206,0.607190,1.484592,1.023718,1.501626
4207,1.218045,1.017098,2.750298,1.100474
4208,1.340217,0.775002,0.504217,-0.912200


### Should remove outliers here

In [166]:
#remove outliers

### Train our best model

In [167]:
from sklearn.ensemble import RandomForestRegressor

rand_forest = RandomForestRegressor(max_depth =  25, max_features =  10, n_estimators = 250)

In [168]:
rand_forest.fit(X_train, y_train)

RandomForestRegressor(max_depth=25, max_features=10, n_estimators=250)

### Predict

In [169]:
y_pred = rand_forest.predict(X_test)

In [170]:
from scipy.special import inv_boxcox

y_pred_salary = inv_boxcox(y_pred, box_cox_transformer)
y_test_salary = inv_boxcox(y_test, box_cox_transformer)

In [171]:
results = pd.DataFrame(zip(y_test_salary, y_pred_salary), columns = ['True Salary', 'Predicted Salary'])
results['Error'] = results['Predicted Salary'] - results['True Salary']

In [172]:
player_team = df[['FULL NAME', 'TEAM']][df['Year'] == 2021]

In [173]:
predicting_2021 = pd.merge(player_team, results, left_index=True, right_index=True)

In [174]:
predicting_2021

,FULL NAME,TEAM,True Salary,Predicted Salary,Error
0,Precious Achiuwa,Tor,2.693401e+06,2.615641e+06,-7.776038e+04
1,Steven Adams,Mem,1.693989e+07,1.246445e+07,-4.475439e+06
2,Bam Adebayo,Mia,2.787475e+07,1.548094e+07,-1.239380e+07
3,LaMarcus Aldridge,Bro,2.624316e+06,9.018845e+06,6.394529e+06
4,Nickeil Alexander-Walker,Nor,3.239581e+06,2.330614e+06,-9.089661e+05
...,...,...,...,...,...
532,Thaddeus Young,San,1.408097e+07,5.774636e+06,-8.306335e+06
533,Trae Young,Atl,8.265438e+06,7.996446e+06,-2.689915e+05
534,Omer Yurtseven,Mia,1.479823e+06,1.898720e+06,4.188967e+05
535,Cody Zeller,Por,2.374079e+06,4.247613e+06,1.873534e+06


In [175]:
#Overpaid players (worst value players)
predicting_2021.sort_values('Error', ascending = True).head(25)

,FULL NAME,TEAM,True Salary,Predicted Salary,Error
503,Kemba Walker,Nyk,3.467769e+07,6.309393e+06,-2.836830e+07
188,Blake Griffin,Bro,3.213895e+07,4.808967e+06,-2.732998e+07
513,Russell Westbrook,Lal,4.383783e+07,1.998634e+07,-2.385149e+07
481,Klay Thompson,Gol,3.766389e+07,1.455367e+07,-2.311022e+07
305,Kevin Love,Cle,3.100159e+07,9.878006e+06,-2.112359e+07
519,Andrew Wiggins,Gol,3.131987e+07,1.340295e+07,-1.791692e+07
200,Tobias Harris,Phi,3.569698e+07,1.900174e+07,-1.669524e+07
133,Goran Dragic,Tor,1.919624e+07,3.315976e+06,-1.588027e+07
437,D'Angelo Russell,Min,2.976788e+07,1.407780e+07,-1.569008e+07
407,Kristaps Porzingis,Dal,3.139045e+07,1.575245e+07,-1.563800e+07


In [176]:
#Underpaid Players (Best value players)
predicting_2021.sort_values('Error', ascending = False).head(25)

,FULL NAME,TEAM,True Salary,Predicted Salary,Error
243,Reggie Jackson,Lac,1.030686e+07,1.766878e+07,7.361917e+06
134,Andre Drummond,Phi,2.385890e+06,9.691409e+06,7.305519e+06
69,Jalen Brunson,Dal,1.790651e+06,8.657813e+06,6.867162e+06
3,LaMarcus Aldridge,Bro,2.624316e+06,9.018845e+06,6.394529e+06
469,Jae'Sean Tate,Hou,1.508541e+06,7.591026e+06,6.082485e+06
406,Bobby Portis,Mil,4.317591e+06,1.030649e+07,5.988898e+06
12,Carmelo Anthony,Lal,2.624316e+06,8.181101e+06,5.556785e+06
171,Shai Gilgeous-Alexander,Okc,5.456754e+06,1.088543e+07,5.428676e+06
493,Jonas Valanciunas,Nor,1.389256e+07,1.928336e+07,5.390799e+06
402,Jakob Poeltl,San,8.685575e+06,1.318999e+07,4.504414e+06


In [177]:
#Best Predicted players
predicting_2021['Absolute Error'] = abs(predicting_2021['Error'])
predicting_2021.sort_values('Absolute Error', ascending = True).head(25)

,FULL NAME,TEAM,True Salary,Predicted Salary,Error,Absolute Error
129,Damyean Dotson,Nyk,2.046427e+05,2.036818e+05,-960.842779,960.842779
314,Naji Marshall,Nor,1.508541e+06,1.514576e+06,6034.734946,6034.734946
460,Cassius Stanley,Det,2.554263e+05,2.624133e+05,6986.975612,6986.975612
500,Dean Wade,Cle,1.771351e+06,1.761892e+06,-9458.949282,9458.949282
531,Moses Wright,Lac,1.719830e+05,1.818242e+05,9841.253680,9841.253680
491,Rayjon Tucker,Den,1.909173e+05,2.013447e+05,10427.412578,10427.412578
303,Robin Lopez,Orl,4.965029e+06,4.975529e+06,10499.406462,10499.406462
509,Tremont Waters,Tor,1.909173e+05,1.746427e+05,-16274.626762,16274.626762
79,Facundo Campazzo,Den,3.178553e+06,3.159637e+06,-18916.478512,18916.478512
212,Haywood Highsmith,Mia,5.446513e+05,5.212636e+05,-23387.726285,23387.726285


In [178]:
#Group by team
team_predictions = predicting_2021.groupby('TEAM').sum().reset_index()

### The fact that just about every team has a lower predicted salary tells me we need to scale 2021 to a full season... 

In [179]:
#Sorted by most overpaid/underperforming teams.  Looks about right.  
team_predictions.sort_values('Error', ascending = True)

,TEAM,True Salary,Predicted Salary,Error,Absolute Error
2,Bro,1.920551e+08,1.260576e+08,-6.599750e+07,9.117982e+07
19,Nyk,1.328688e+08,7.386789e+07,-5.900091e+07,6.388213e+07
5,Cle,1.326024e+08,7.520999e+07,-5.739241e+07,6.150841e+07
17,Min,1.360817e+08,7.968783e+07,-5.639386e+07,6.791685e+07
9,Gol,1.654710e+08,1.130024e+08,-5.246866e+07,8.318038e+07
13,Lal,1.723392e+08,1.201249e+08,-5.221432e+07,8.672689e+07
0,Atl,1.307366e+08,7.961259e+07,-5.112404e+07,5.848991e+07
28,Uta,1.527846e+08,1.027569e+08,-5.002770e+07,5.720410e+07
27,Tor,1.201630e+08,7.087927e+07,-4.928378e+07,5.210255e+07
1,Bos,1.236814e+08,7.566965e+07,-4.801171e+07,4.946043e+07
